# Project: Wrangle Open Street Map data

In this project I wrangle and extract information from an Open Street Map database (OSM). I chose to analyze the region of Cambridgeshire because Cambridge is a charming town with beautiful surroundings that I enjoyed so much. 

I worked with the full data set available in [Geofabrik](https://download.geofabrik.de/europe/great-britain/england/cambridgeshire.html). Since the uncompressed OSM file has a weight of 70 Mb the most reasonable way to clean and process the data is programmatically. I employed the code written during the "Case Study: OpenStreetMap Data [SQL]" lesson as starting point. Several modifications and improvements were added to the code such as the ```clean.py``` and ```create_db.py``` scripts which were used to correct the data and produce the database ```Cambridge.db``` respectively. 

The OSM dataset is formed by two different elements: nodes and ways. Briefly, a node represents a particular point in the map characterized by latitude and longitude (among other attributes). It can just mark a point in the map with no content or describe some element/s of interest (for example, a building). In the latter case, the node will contain child tags formed by a pair of key and value which detail the properties of the node (for example, the different fields of the address). 

This is an example of a node taken from ```Cambridge.osm```. It shows the geographic position (latitude, longitude) of a traffic signal in a highway.

```xml
<node id="104521" lat="52.2035316" lon="0.123448" version="1">
    <tag k="highway" v="traffic_signals"/>
```

On the other hand, ways are a collection of nodes that can be visualized, for example, as a path in the map area. The child tag elements <span style="color:green">nd</span> list all the nodes in the way, whereas the tags reflect information about different fields. For example, this way represents a highway with speed limit of 70 mph (among other details).

```xml
<way id="1881406" version="1">
    <nd ref="608754"/>
    <nd ref="620848"/>
    ...
    <tag k="oneway" v="yes"/>
    <tag k="highway" v="trunk"/>
    <tag k="maxspeed" v="70 mph"/>
    <tag k="operator" v="Highways England"/>
    ...
</way>
```

This project is organized as follows: in Section 1 I provide a description of the problems found in the dataset and explain how they were corrected. In Section 2 I perform SQL queries on the clean data which are intended to get specific information from the data. Finally, I conclude in Section 3 summarizing the data wrangling and proposing several improvements to this work.

## Section 1:   Problems in the original dataset 

The OSM dataset contains a lot of human entered data. It is normal then to encounter uncomplete data or errors in some fields. A quick look to the OSM file revealed the following problems:

* Tags with non ascii characters
* Non uniform key value format
* Wrong postal codes
* Telephone number format
* Recycling amenity format
* Wrong names or non uniform word abbreviations

Due to the size of the OSM data file it is not possible to read all the  content at once. In order to avoid memory problems we use the ```xml.etree.cElementT``` module to read one tag at a time. The cleaning process is done element by element and it is implemented in the ```shape_element``` definition in ```process_data.py```, where the methods from ```clean.py``` are used to address the aforementioned problems. 

After this is done, the data is stored in csv files as follows: for the nodes, we separate the information about the nodes attributes (written in ```nodes.csv```) from the nodes child tags (written in ```nodes_tags.csv```); for the ways, there is an extra file with the extra file ```ways_nodes.csv``` which lists the nodes contained in the paths. In addition, the each csv file is double-checked with the ```validate_element``` method (by default is set to False due to the computation time) to ensure that there is no missing data or errors in the structure. The corresponding schemas of the output files can be found in ```schema.py```. Finally the csv files are merged in the ```Cambridge.db``` database with ```create_db.py``` for the later SQL analysis.

### Tags with non ASCII characters

The ```Cambridge.osm``` file is encoded with UTF-8. In principle, this would not represent any problem but I decided to restrict the encoding to ASCII for the sake of simplicity (I can not understand data written written, for example, Russian or Chineese characters). To do so, first I identified the tags with non-ASCII symbols and, if possible, I subtituted them with close characters. For example: "ä" is converted into "ae" and "\`" is converted to "'". Those tags in which this is not possible are removed. This correction implemented with the ```audit_nonascii``` definition writen in the ```clean.py``` script.

### Non uniform  key value formats

As explained before, the child tags contain relevant information about a particular node or way. The field key labels a particular attribute, like address or amenity, and the field value contains its information. The problem with this arises when keys representing the same field do not have the same format, like "postal_code" and "postcode" or they present the data in a non-intuitive way, like the following node


```xml
<node id="20922173" lat="52.2072198" lon="0.1219277" version="1">
    <tag k="name" v="d&#39;Arry&#39;s Liquor Loft and Restaurant"/>
    <tag k="amenity" v="pub"/>
    <tag k="fhrs:id" v="506819"/>
    <tag k="website" v="http://www.darrys.co.uk"/>
    <tag k="old_name" v="Cambridge Arms"/>
    <tag k="addr:city" v="Cambridge"/>
    <tag k="addr:street" v="King Street"/>
    <tag k="source:addr" v="FHRS Open Data"/>
    <tag k="addr:postcode" v="CB1 1LN"/>
    <tag k="addr:housenumber" v="2-4"/>
```

As we can see, the keys of the information related to the address have the format "< source > : < field >". To make the access to the data more friendly, I split the original key field value into two parts and updated the key value with "< field >". A new element called type was created to store the remaining "< source >" value. This is implemented automaticaly while reading the data in ```process_data.py```.

### Wrong Postal Codes

The [postal codes](https://en.wikipedia.org/wiki/CB_postcode_area) in the Cambridgeshire area are formed by two blocks: the first one is composed by the letters CB followed by a number (1-11) or (21-25), whereas the second one is a combination of three characters (letters and/or numbers). This variable was stored in the OSM data under the key label "postal_code" or "postcode". I corrected this by setting "postal_code" as key. Also, those postal codes with missing/extra numbers were eliminated from the dataset. The ```c_pc``` definition in the ```clean.py``` was coded for this purpose.

### Telephone number format

I also found problems with the format of the telephone numbers. Some of them included the country calling code prefix +44, while other missed it and had the trunk prefix 0 instead  of having the numbers grouped in different ways. I fixed this by arranging the format number into +44XXXXXXXXXX. Those with missing/extra digits were eliminated from the dataset. This is implemented with the ```c_telephone``` definition in the ```clean.py``` script.

### Recicling amenity format

Some nodes represented buildings/places which child tags contained information about waste management. Even when the data seemed to be correct, it was difficult to read due to the amount of recycling items considered. I grouped these elements into five groups: plastic, tin, glass, paper and general. The definition ```c_recycling``` in ```clean.py``` takes care of this errors.

### Wrong names or non uniform word abbreviations

I also found that some words had different abbreviations or were spelled incorrectly. I correct this with the ```c_tagstr``` definition in ```clean.py``` script.

## Section 2: Quering data with SQL

Once the OSM is parsed, corrected and stored into the csv files one can think about getting information from it. To do so, the output files are stored into a database (Cambdridge.db). This was implemented with the ```create_db.py``` script making use of the pandas method [```DataFrame.to_sql```](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html). I chose SQL as the query language to get results from the database.

In [7]:
# LOAD SQL MODULE 
%load_ext sql
# LOAD Cambdridge.db DATABASE
%sql sqlite:///Cambridge.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


u'Connected: None@Cambridge.db'

I performed the following queries in order to obtain a general idea about some fields in the database. They are also formulated as questions to improve the readability of the report.

* How many nodes/ways are there in the database?

As we can see from the queries below, the number of nodes and ways in the dataset are 528955 and 97429 respectively.

In [8]:
%sql select count(*) as "Number of Nodes" from nodes;

Done.


Number of Nodes
528955


In [9]:
%sql select count(*) as "Number of Ways" from ways;

Done.


Number of Ways
97429


* Which are the most common amenities?

We can search in the data for the elements with "amenity" as key and then grouping their corresponding values. The query below shows the top 10 amenities present in the nodes_tags dataset.

In [10]:
%%sql 
select count(*) as "number of amenities", nodes_tags.value 
from nodes_tags where nodes_tags.key="amenity" 
group by nodes_tags.value 
order by "number of amenities" desc limit 10;

Done.


number of amenities,value
848,bicycle_parking
518,bench
358,post_box
296,parking
148,restaurant
123,cafe
97,recycling
90,fast_food
77,pub
50,telephone


* How many places of worship are there? and which are the top 10 major religion confessions/denominations?

Querying the places of worship is done in the same way than the previous question but this time the answer is completely different from the nodes_tags and ways_tags tables. The nodes_tags only contains information about christian and muslim religions, while the ways_tags query displays more information: Christian, Jewish, Buddhist... The data appears to be different since the Mulsim religion only has one mosque instead of the two shown in the nodes_tags table. However, the answer for the first question is the same, the christian religion is a majority in this dataset in terms of places of worship.

In [11]:
%%sql 
select count(*), *
from nodes_tags where nodes_tags.key="religion"
group by nodes_tags.value;

Done.


count(*),index,id,key,value,type
9,102035,4209178250,religion,christian,regular
2,33012,1343651837,religion,muslim,regular


In [12]:
%%sql 
select count(*) as "number", ways_tags.value from ways_tags
where ways_tags.key = "religion"
group by ways_tags.value
order by "number" desc;

Done.


number,value
167,christian
3,jewish
1,buddhist
1,hindu
1,muslim
1,sikh
1,unitarian


To answer the second question I considered querying the ways_tags table since it contains more religions which seems a more complete data for this field. As expected in England, the Anglican church represents the majority of the religious confessions.

In [14]:
%%sql 
select count(*) as "number", ways_tags.value from ways_tags
where ways_tags.key = "denomination"
group by ways_tags.value
order by "number" desc limit 10;

Done.


number,value
89,anglican
15,baptist
9,methodist
6,united_reformed
3,catholic
3,independent
2,roman_catholic
2,salvation_army
1,Dominican
1,Jewish


* How many rubbish bins/recycling places are there? and for what type of garbage (glass, plastic...) are there more rubbish bins available?

To answer the first question we need to look for elements with "recycling" as tag value in the nodes_tags table. We find a total of 97 entries.

In [15]:
%%sql 
select count(*) from nodes_tags
where nodes_tags.key = "amenity" and nodes_tags.value = "recycling";

Done.


count(*)
97


In [16]:
%%sql 
select count(*) as "counts", nodes_tags.key from nodes_tags
where nodes_tags.type = "recycling" and nodes_tags.value = "yes"
group by nodes_tags.key 
order by "counts" desc;

Done.


counts,key
34,glass
29,paper
19,plastic
16,general
8,tin
6,batteries


## Conclusions 

During this project we have parsed an OSM file with human-entered data. In order to extract some meaningful information about the dataset we looked for inconsistencies/errors in the dataset. Once this was done, we programmatically corrected the data and stored it in a database. In order to get an insight of it, SQL queries were performed to investigate such things as: number of nodes/ways, religion denominations and temples of worship, amenities and recycling places.

## Improvements to this work

Perhaps the better improvement that one could do to this work is to parse and correct every field of every way and node. By doing this, we will ensure that we have all the data in a uniform format. To standardize every field programmatically would represent a really considerably effort. One would have to write at least one python method per field to do this. For the OSM employed in this work this would mean 381 methods to account all the keys only present in the nodes child tags.

In [17]:
%sql select count(distinct nodes_tags.key) as "Different nodes tags" from nodes_tags;

Done.


Different nodes tags
381


Also, the method ```c_tagstr``` in ```clean.py``` would have to be extended to check the sintax of every relevant word in the dataset (keys and values of the child tags). Notice that in this work we only corrected a few fields that I notice while studying the data base. Detect all the possible mistakes in so many fields would represent a challenge. 

If the latter improvement can be achieved, we could extract the complete standarized  information from the data set and with SQL or other query languages we could get information about the whole area. One could extend the results presented in this project to obtain information about any field. Statistics about stablishments, highways, etc...

## Bibliography

Throughout this work I found very helpfull the following sources, both for the development of the codes and to write the report. 

* https://regex101.com
* https://wiki.openstreetmap.org/wiki/OSM_XML
* https://github.com/jkarakas/Wrangle-OpenStreetMaps-Data-with-SQL
* https://gist.github.com/carlward/54ec1c91b62a5f911c42#file-sample_project-md
* https://discussions.udacity.com/c/nd002-data-wrangling/nd002-p-wrangle-openstreetmaps-data-with-sql